In [ ]:
%matplotlib inline
import matplotlib.image as mpimg
import numpy as np
import matplotlib.pyplot as plt
import os,sys
from scipy import ndimage
from PIL import Image
from sklearn.cluster import KMeans
from sklearn import linear_model
from sklearn import svm
import cv2

In [ ]:
# Helper functions
def load_image(infilename):
    data = mpimg.imread(infilename)
    return data

def img_float_to_uint8(img):
    rimg = img - np.min(img)
    rimg = (rimg / np.max(rimg) * 255).round().astype(np.uint8)
    return rimg

# Concatenate an image and its groundtruth
def concatenate_images(img, gt_img):
    nChannels = len(gt_img.shape)
    w = gt_img.shape[0]
    h = gt_img.shape[1]
    if nChannels == 3:
        cimg = np.concatenate((img, gt_img), axis=1)
    else:
        gt_img_3c = np.zeros((w, h, 3), dtype=np.uint8)
        gt_img8 = img_float_to_uint8(gt_img)          
        gt_img_3c[:,:,0] = gt_img8
        gt_img_3c[:,:,1] = gt_img8
        gt_img_3c[:,:,2] = gt_img8
        img8 = img_float_to_uint8(img)
        cimg = np.concatenate((img8, gt_img_3c), axis=1)
    return cimg

def img_crop(im, w, h):
    list_patches = []
    imgwidth = im.shape[0]
    imgheight = im.shape[1]
    is_2d = len(im.shape) < 3
    for i in range(0,imgheight,h):
        for j in range(0,imgwidth,w):
            if is_2d:
                im_patch = im[j:j+w, i:i+h]
            else:
                im_patch = im[j:j+w, i:i+h, :]
            list_patches.append(im_patch)
    return list_patches

def value_to_class(v):
    df = np.sum(v)
    if df > foreground_threshold:
        return 1
    else:
        return 0

In [ ]:
def rotation(orig, gts):
    ks=[90,180,270]
    rotated=[ndimage.rotate(img,k) for img in orig for k in ks]
    gt_rotated=[ndimage.rotate(gt_img,k) for gt_img in gts for k in ks]
    orig=orig+rotated
    gts=gts+gt_rotated
    print(len(orig))
    print(len(gts))
    return orig,gts

def add_gray_dimension(img):
    out=np.dot(img[...,:3], [0.299, 0.587, 0.114])
    shape_one=[out.shape[0], out.shape[1], 1]
    out = np.reshape(out, shape_one)
    return out

def add_laplacian(img):
    laplbew=ndimage.filters.laplace(add_gray_dimension(img))
    lapl=ndimage.filters.laplace(img)
    return laplbew,lapl

def add_sobel(img):
    sx = ndimage.sobel(img, axis=0, mode='constant')
    sy = ndimage.sobel(img, axis=1, mode='constant')
    sob = np.hypot(sx, sy)
    return sob

def add_segment(im):
    n = 10
    l = 256
    im = ndimage.gaussian_filter(im, sigma=l/(4.*n))
    mask = (im > im.mean()).astype(np.float)
    mask += 0.1 * im
    img = mask + 0.2*np.random.randn(*mask.shape)
    hist, bin_edges = np.histogram(img, bins=60)
    bin_centers = 0.5*(bin_edges[:-1] + bin_edges[1:])
    binary_img = img > 0.5
    open_img = ndimage.binary_opening(binary_img)
    # Remove small black hole
    close_img = ndimage.binary_closing(open_img)
    close_img=add_gray_dimension(close_img)
    return close_img

In [ ]:
def extract_features(img):
    gray_img = add_gray_dimension(img)
    sob = add_sobel(img)
    lapbew,lap=add_laplacian(img)
    seg=add_segment(img)
    img = np.concatenate((img, gray_img), axis = 2)
    img = np.concatenate((img, sob), axis = 2)
    img = np.concatenate((img, lapbew), axis = 2)
    img = np.concatenate((img, lap), axis = 2)    
    img = np.concatenate((img, seg), axis = 2)
    
    feat_m = np.mean(img, axis=(0,1))
    feat_v = np.var(img, axis=(0,1))
    feat = np.append(feat_m, feat_v)
    return feat

In [ ]:
# Obtain train set
# Loaded a set of images
foreground_threshold = 0.25 # percentage of pixels > 1 required to assign a foreground label to a patch
root_dir = "training/"
image_dir = root_dir + "images/"
files = os.listdir(image_dir)
n = min(75, len(files)) # Load maximum 20 images
imgs = [load_image(image_dir + files[i]) for i in range(n)]
gt_dir = root_dir + "groundtruth/"
gt_imgs = [load_image(gt_dir + files[i]) for i in range(n)]
imgs,gt_imgs = rotation(imgs,gt_imgs)

patch_size = 16 # each patch is 16*16 pixels
img_patches = [img_crop(imgs[i], patch_size, patch_size) for i in range(len(imgs))]
gt_patches = [img_crop(gt_imgs[i], patch_size, patch_size) for i in range(len(gt_imgs))]
img_patches = np.asarray([img_patches[i][j] 
                          for i in range(len(img_patches)) 
                          for j in range(len(img_patches[i]))])
gt_patches =  np.asarray([gt_patches[i][j] 
                          for i in range(len(gt_patches)) 
                          for j in range(len(gt_patches[i]))])

X = np.asarray([ extract_features(img_patches[i]) for i in range(len(img_patches))])
Y = np.asarray([value_to_class(np.mean(gt_patches[i])) for i in range(len(gt_patches))])

In [ ]:
logreg = linear_model.LogisticRegression(C=1e5, class_weight="balanced")
logreg.fit(X, Y)
Z=logreg.predict(X)
print('F1_score = ' + str(f1_score(Y, Z)))

In [ ]:
clf = svm.SVC(gamma='auto')
clf.fit(X,Y)
Z=clf.predict(X)
print('F1_score = ' + str(f1_score(Y, Z)))

In [ ]:
# Obtain test set
root = "training/"
image_dir = root + "images/"
files = os.listdir(image_dir)
imgs_te = [load_image(image_dir + files[i]) for i in np.arange(n+1,len(files))]
gt_dir = root_dir + "groundtruth/"
gt_imgs_te = [load_image(gt_dir + files[i]) for i in np.arange(n+1,len(files))]

img_patches_te = [img_crop(imgs_te[i], patch_size, patch_size) for i in range(len(imgs_te))]
gt_patches_te = [img_crop(gt_imgs_te[i], patch_size, patch_size) for i in range(len(gt_imgs_te))]
img_patches_te = np.asarray([img_patches_te[i][j] 
                             for i in range(len(img_patches_te)) 
                             for j in range(len(img_patches_te[i]))])
gt_patches_te =  np.asarray([gt_patches_te[i][j] 
                             for i in range(len(gt_patches_te)) 
                             for j in range(len(gt_patches_te[i]))])
X_te = np.asarray([extract_features(img_patches_te[i]) for i in range(len(img_patches_te))])
Y_te = np.asarray([value_to_class(np.mean(gt_patches_te[i])) for i in range(len(gt_patches_te))])

Z_te = logreg.predict(X_te)
print('F1_score = ' + str(f1_score(Y, Z_te)))

Z_te = clf.predict(X_te)
print('F1_score = ' + str(f1_score(Y, Z_te)))